# Prompts

In this notebook I manage the prompt templates. It saves it in a structured way using the PromptManager class. 

In [ ]:
from src.prompt_manager import PromptManager_old, create_prompt_dict, create_chat_prompt_dict, PromptTemplate
from pathlib import Path

TODO: use metadata that better describes prompts, e.g. categories for "few-shot 0/1", "BARS 0/1", ...


# Barter Deals

In [11]:
import pandas as pd
df = pd.read_parquet('../data/raw/BARTER_DEALS.parquet')

In [12]:
df

,created_at,updated_at,deleted_at,id,deal_id,deal_title,deal_text,partner_id,locations,deal_type,...,last30_days_cancelled_applications_count,accepts_international,accepted_countries,min_social_media_followers,live_since,live_until,deal_id_str,accepted_countries_str,first_content_type,applications_till_day_7
0,2025-05-20 13:00:23.364737,2025-11-05 06:37:43.458654,NaT,3589,0196edc7-e6a8-00c8-6ff5-ea60fa3f2895,Paid deal: Moco Museum shoot (May 23 – €200),Moco Museum Amsterdam is inviting a small grou...,85,"[{'address': 'Singel, Amsterdam, Netherlands',...",physical,...,0,False,[NLD],2500,2025-05-20 15:21:28.249567,NaT,0196edc7-e6a8-00c8-6ff5-ea60fa3f2895,NLD,Activities,39
1,2025-05-01 05:38:46.147207,2025-05-27 06:47:49.284494,NaT,2803,019689e5-f489-00c8-8c26-06b3ea0961a5,Promotion clothing,Sharing unique and HQ content photos (one post...,153,None,online,...,0,False,[NLD],5000,2023-11-28 14:59:51.626780,NaT,019689e5-f489-00c8-8c26-06b3ea0961a5,NLD,Fashion,0
2,2025-05-01 05:36:56.354872,2025-06-20 06:33:33.336563,NaT,622,019689e2-bfb5-00c8-6485-23352d64b0e5,Matrastopper,Wij zijn op zoek naar creatieve en grappige in...,1102,None,online,...,0,False,[NLD],5000,2025-03-04 11:46:18.176627,NaT,019689e2-bfb5-00c8-6485-23352d64b0e5,NLD,UGC,13
3,2025-05-01 05:37:43.926310,2025-08-11 19:54:20.702222,NaT,1527,019689e3-d7a5-00c8-f5a8-6bfe51a21cd6,FREE Oversized VIBE T-SHIRT from Vibecreation.nl,Vibecreation (VIBE) isn’t just a brand—it’s an...,1162,None,online,...,0,False,[NLD],2500,2025-04-17 09:50:45.518632,NaT,019689e3-d7a5-00c8-f5a8-6bfe51a21cd6,NLD,Fashion,19
4,2025-05-01 05:38:37.126799,2025-11-05 06:31:29.851951,NaT,2614,019689e5-7dd0-00c8-615b-7750a8fec2f3,Ready-To-Blend Protein Smoothies targeted at c...,You will receive three Ready-To-Blend Smoothie...,1118,"[{'address': 'Valentijnkade 57C, 1095 JK Amste...",physical,...,0,False,[NLD],2500,2025-02-10 18:19:51.240203,NaT,019689e5-7dd0-00c8-615b-7750a8fec2f3,NLD,Sport,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5104,2025-05-01 05:38:27.299422,2025-11-05 06:30:09.954042,NaT,2431,019689e5-0296-00c8-288a-3562dd9c5f04,Food | Local Taste Tilburg | 10 proeverijen | ...,Hé Bourgondiër!\n\nTof dat je nadenkt om met d...,647,"[{'address': '5038 WN Tilburg, Netherlands', '...",physical,...,0,False,[NLD],5000,2024-10-13 10:00:59.618897,NaT,019689e5-0296-00c8-288a-3562dd9c5f04,NLD,Activities,26
5105,2025-05-01 05:38:32.541901,2025-11-05 06:30:52.552884,NaT,2532,019689e5-45dc-00c8-2f02-58bf11ea15d8,Deel jouw Funky Falcon moment!,"Wij zijn Two Chefs Brewing, een Amsterdamse cr...",459,"[{'address': 'Moezelhavenweg 6, 1043 AM Amster...",physical,...,0,False,[NLD],5000,2024-07-18 14:36:08.008194,NaT,019689e5-45dc-00c8-2f02-58bf11ea15d8,NLD,Drinks,7
5106,2025-05-01 05:38:32.589051,2025-11-05 06:30:53.054004,NaT,2533,019689e5-4642-00c8-08dd-41d776586f1c,Private Museum Visit at Nxt Museum,Bring a friend and have the the entire museum ...,365,"[{'address': 'Asterweg, 1031 Amsterdam, Nether...",physical,...,0,False,[NLD],50000,2024-03-26 14:09:02.539962,NaT,019689e5-4642-00c8-08dd-41d776586f1c,NLD,Experiences,5
5107,2025-05-01 05:38:35.681583,2025-11-05 06:31:18.352139,NaT,2587,019689e5-6ce0-00c8-5fdd-a058811d2946,Chocomates pictures,We are looking to promote our Chocomates produ...,661,"[{'address': 'Verlengde Spiegelmakerstraat 51,...",physical,...,0,False,[NLD],10000,2024-10-17 14:59:27.484092,NaT,019689e5-6ce0-00c8-5fdd-a058811d2946,NLD,Experiences,7


In [13]:
pm = PromptManager(folder=Path("../prompts/BARTER_DEALS/"))

USER_PROMPT_TEMPLATE = """Deal text:
"{deal_text}"
"""

nr_of_output_tokens = "tokens_5"


## Naive prompt

In [14]:
SYSTEM_PROMPT = """
You will be given a deal text.
Your task is to rate the overall quality of the deal text on a scale from 1 to 5.
Only output the integer rating.
"""

In [ ]:
template_chat = {
    "system": SYSTEM_PROMPT.strip(),
    "user": USER_PROMPT_TEMPLATE.strip()}


chat_prompt_dict = create_chat_prompt_dict(
    name="naive_judge",
    description="Base evaluation prompt that requests rating.",
    template_chat=template_chat,
    constrained_output=["1", "2", "3", "4", "5"],
    tags=[nr_of_output_tokens, "QA_evaluation", "baseline"]
)

pm.save_prompt(chat_prompt_dict)

WindowsPath('../prompts/BARTER_DEALS/naive_judge.yml')

## BARS style prompt

In [16]:
SYSTEM_PROMPT = """
You will be given a deal text that describes a collaboration opportunity for creators or influencers.
Your task is to provide a 'total rating' scoring how appealing, clear, and persuasive the deal text is for its intended audience.

Give your answer as an integer on a scale of 1 to 5, where:
1: means the deal text is poorly written, unclear, or unappealing.
2: means the deal text is somewhat understandable but not persuasive or engaging.
3: means the deal text is clear and moderately appealing.
4: means the deal text is strong, clear, and persuasive.
5: means the deal text is excellent — very clear, engaging, and convincing.
"""

In [17]:
template_chat = {
    "system": SYSTEM_PROMPT.strip(),
    "user": USER_PROMPT_TEMPLATE.strip()}

chat_prompt_dict = create_chat_prompt_dict(
    name="BARS_judge_12345",
    description="BARS style prompt.",
    template_chat=template_chat,
    constrained_output=["1", "2", "3", "4", "5"],
    tags=[nr_of_output_tokens, "QA_evaluation", "BARS"]
)
pm.save_prompt(chat_prompt_dict)

WindowsPath('../prompts/BARTER_DEALS/BARS_judge_12345.yml')

# Q&A Evaluation (McGill dataset)

In [ ]:
pm = PromptManager(folder="../prompts/QA_McGill")

In [3]:
USER_PROMPT_TEMPLATE = """Question:
"{question}"

Answer:
"{answer}"
"""

## Five output tokens

In [4]:
nr_of_output_tokens = "tokens_5"

## naive_prompt

In [63]:
SYSTEM_PROMPT = """You will be given a user_question and system_answer couple.
Your task is to provide a 'total rating' scoring how well the system_answer answers the user concerns expressed in the user_question.
Give your answer as an integer on a scale of 1 to 5, where 1 means that the system_answer is not helpful at all, and 5 means that the answer completely and helpfully addresses the question."""

In [13]:
template_chat = {
    "system": SYSTEM_PROMPT.strip(),
    "user": USER_PROMPT_TEMPLATE.strip()}


chat_prompt_dict = create_chat_prompt_dict(
    name="Judge v1",
    description="Base evaluation prompt that requests rating.",
    template_chat=template_chat,
    constrained_output=["1", "2", "3", "4", "5"],
    tags=[nr_of_output_tokens, "QA_evaluation", "baseline"]
)

pm.save_prompt(chat_prompt_dict)

WindowsPath('../prompts/Barter_deals/Judge v1.yml')

## BARS_prompt

In [68]:
SYSTEM_PROMPT = """
You will be given a user_question and system_answer couple.
Your task is to provide a 'total rating' scoring how well the system_answer answers the user concerns expressed in the user_question.
Give your answer as an integer on a scale of 1 to 5, where 1 means that the system_answer is not helpful at all, and 5 means that the answer completely and helpfully addresses the question.

Here is the scale you should use to build your answer:
1: The system_answer is terrible: completely irrelevant to the question asked, or very partial.
2: The system_answer is mostly not helpful: misses key aspects of the question or misunderstands the user’s intent.
3: The system_answer is mostly helpful: provides useful information but lacks depth, precision, or structure.
4: The system_answer is excellent: relevant, direct, detailed, and addresses all the concerns raised in the question.
5: The system_answer is outstanding: not only complete and precise, but also insightful, well-structured, and engaging — it exceeds expectations by adding clarity, nuance, or empathy.
"""

In [ ]:
chat_prompt_dict = create_chat_prompt_dict(
    name="Judge v2",
    description="BARS style prompt.",
    system_message=SYSTEM_PROMPT,
    user_message_template=USER_PROMPT_TEMPLATE,
    constrained_output=["1", "2", "3", "4", "5"],
    tags=[nr_of_output_tokens, "QA_evaluation", "BARS"]
)
pm.save_prompt(chat_prompt_dict)

WindowsPath('../prompts/Judge v2.yml')

In [5]:
SYSTEM_PROMPT = """
You will be given a user_question and system_answer couple.
Your task is to provide a 'total rating' scoring how well the system_answer answers the user concerns expressed in the user_question.
Give your answer as using A, B, C, D, or E, where A means that the system_answer is not helpful at all, and E means that the system_answer completely and helpfully addresses the question.

Here is the scale you should use to build your answer:
A: The system_answer is terrible: completely irrelevant to the question asked, or very partial.
B: The system_answer is mostly not helpful: misses key aspects of the question or misunderstands the user’s intent.
C: The system_answer is mostly helpful: provides useful information but lacks depth, precision, or structure.
D: The system_answer is excellent: relevant, direct, detailed, and addresses all the concerns raised in the question.
E: The system_answer is outstanding: not only complete and precise, but also insightful, well-structured, and engaging — it exceeds expectations by adding clarity, nuance, or empathy.
"""

In [ ]:
# chat_prompt_dict = create_chat_prompt_dict(
#     name="Judge v2_ABCDE",
#     description="BARS style prompt.",
#     # system_message=SYSTEM_PROMPT,
#     # user_message_template=USER_PROMPT_TEMPLATE,
#     constrained_output=["1", "2", "3", "4", "5"],
#     tags=[nr_of_output_tokens, "QA_evaluation", "BARS"]
# )
# pm.save_prompt(chat_prompt_dict)

TypeError: create_chat_prompt_dict() missing 1 required positional argument: 'template_chat'

In [12]:
template_chat = {
    "system": SYSTEM_PROMPT.strip(),
    "user": USER_PROMPT_TEMPLATE.strip()}

chat_prompt_dict = create_chat_prompt_dict(
    name="Judge v2_ABCDE",
    description="BARS style prompt.",
    template_chat=template_chat,
    constrained_output=["A", "B", "C", "D", "E"],
    tags=[nr_of_output_tokens, "QA_evaluation", "BARS"]
)
pm.save_prompt(chat_prompt_dict)

WindowsPath('../prompts/Judge v2_ABCDE.yml')

In [9]:
chat_prompt_dict

{'id': 'bf77b2f9f8',
 'name': 'Judge v2_ABCDE',
 'description': 'BARS style prompt.',
 'constrained_output': ['A', 'B', 'C', 'D', 'E'],
 'tags': ['tokens_5', 'QA_evaluation', 'BARS'],
 'version': 1,
 'template_chat': {'system': 'Rating: ',
  'user': 'Question:\n"{question}"\n\nAnswer:\n"{answer}"'}}

## BARS_prompt with Role Instruction

In [72]:
SYSTEM_PROMPT = """
You are an expert evaluator trained to rate how helpful a system's answer is, given a user's question.
Give your answer as an integer on a scale of 1 to 5, where 1 means that the system_answer is not helpful at all, and 5 means that the answer is outstandingly helpful.

Use the following 5-point scale:
1 — Terrible: completely irrelevant or severely misunderstands the question.
2 — Mostly not helpful: partially relevant but misses key aspects or contains major errors.
3 — Mostly helpful: somewhat useful, but lacks depth, precision, or structure.
4 — Excellent: relevant, accurate, and detailed; addresses all main points.
5 — Outstanding: exceptionally clear, insightful, and comprehensive; exceeds expectations.
"""

In [ ]:
chat_prompt_dict = create_chat_prompt_dict(
    name="Judge v3",
    description="BARS with role instruction.",
    system_message=SYSTEM_PROMPT,
    user_message_template=USER_PROMPT_TEMPLATE,
    constrained_output=["1", "2", "3", "4", "5"],
    tags=[nr_of_output_tokens, "evaluation", "BARS", "role_instruction"]
)
pm.save_prompt(chat_prompt_dict)

WindowsPath('../prompts/Judge v3.yml')

## BARS_prompt, on categorical scale

This prompt template is currently not being used.

In [76]:
nr_of_output_tokens = "undefined"

In [77]:
SYSTEM_PROMPT = """
You are an expert evaluator trained to rate how helpful a system's answer is, given a user's question.
Your task is to provide a rating of the helpfulness of the system_answer, given the user_question.

Use the following scale:
Terrible: completely irrelevant to the question asked, or very partial.
Mostly not helpful: misses key aspects of the question or misunderstands the user’s intent.
Mostly helpful: provides useful information but lacks depth, precision, or structure.
Excellent: relevant, direct, detailed, and addresses all the concerns raised in the question.
Outstanding: not only complete and precise, but also insightful, well-structured, and engaging — it exceeds expectations by adding clarity, nuance, or empathy.
"""

In [ ]:
chat_prompt_dict = create_chat_prompt_dict(
    name="Judge v4",
    description="BARS, no integers.",
    system_message=SYSTEM_PROMPT,
    user_message_template=USER_PROMPT_TEMPLATE,
    constrained_output=["1", "2", "3", "4", "5"],
    tags=[nr_of_output_tokens, "evaluation", "BARS", "no_integers"]
)
pm.save_prompt(chat_prompt_dict)

WindowsPath('../prompts/Judge v4.yml')

# Show all existing prompts

In [4]:
pm.load_all()

{'7785d73041': Prompt(id='7785d73041', name='Judge v1', description='Base evaluation prompt that requests rating.', constrained_output=['1', '2', '3', '4', '5'], tags=['tokens_5', 'QA_evaluation', 'baseline'], version=1, system_message="You will be given a user_question and system_answer couple.\nYour task is to provide a 'total rating' scoring how well the system_answer answers the user concerns expressed in the user_question.\nGive your answer as an integer on a scale of 1 to 5, where 1 means that the system_answer is not helpful at all, and 5 means that the answer completely and helpfully addresses the question.", user_message_template='Question:\n"{question}"\n\nAnswer:\n"{answer}"'),
 'ab7a881d07': Prompt(id='ab7a881d07', name='Judge v2', description='BARS style prompt.', constrained_output=['1', '2', '3', '4', '5'], tags=['tokens_5', 'QA_evaluation', 'BARS'], version=1, system_message="You will be given a user_question and system_answer couple.\nYour task is to provide a 'total r